In [3]:
import random
import numpy as np
import pandas as pd

In [4]:
suits = ('Hearts','Diamonds','Spades','Clubs')
ranks = ('Two','Three','Four','Five','Six','Seven','Eight','Nine','Ten','Jack','Queen','King','Ace')
values = {'Two':2,'Three':3,'Four':4,'Five':5,'Six':6,'Seven':7,'Eight':8,'Nine':9,'Ten':10,'Jack':10,'Queen':10,'King':10,'Ace':11}
playing = True

In [5]:
class Card:
       
    def __init__(self,rank,suit):
        self.rank = rank
        self.suit = suit
        
    def __str__(self):
        return f'{self.rank} of {self.suit}'


In [247]:
class Deck:
    
    def __init__(self):
        self.deck = []
        self.count = 0
        
        for suit in suits:
            for rank in ranks:
            
                self.deck.append(Card(rank,suit))
        self.deck = n*self.deck
    
    def __iter__(self):
        return iter(self.deck)
    
    def shuffle(self):
        random.shuffle(self.deck)
        
    def deal(self):
        c = self.deck.pop()
        if c.rank in ranks[:5]:
            self.count += 1
        if c.rank in ranks[8:]:
            self.count -=1
        self.deck.pop()
        return c

In [28]:
class Hand:
    def __init__(self):
        self.cards = []  
        self.value = 0   
        self.aces = 0
    
    def add_card(self,card):
        self.cards.append(card)        
                                        
        self.value += values[card.rank]
        if card.rank == 'Ace':
            self.aces += 1
   
    def adjust_for_ace(self):
        while self.value > 21 and self.aces:
            self.value -= 10
            self.aces -= 1

In [29]:
def hit(deck,hand):
    hand.add_card(deck.deal())
    hand.adjust_for_ace()

In [30]:
def hit_or_stand(deck, hand, action):
    global playing

    while True:
        if action == 'stand':
            playing = False
        elif action == 'hit':
            hit(deck,hand)
        break

In [31]:
n = int(input('how many decks?'))
the_deck = Deck()
the_deck.shuffle()
score = 0

while True:    
    
    print(f'Your current score is {score}')

    player_hand = Hand()
    
    player_hand.add_card(the_deck.deal())

    player_hand.add_card(the_deck.deal())
    
    player_hand.adjust_for_ace()

    print("\nPlayer's Hand:", *player_hand.cards, sep = '\n')
    print("\nPlayer's Hand =", player_hand.value)
    

    while playing:
        
        if len(list(the_deck)) == 0:
            print('no cards left!')
            break 
  
        if player_hand.value >= 17:
            action = 'stand'
           
        
        else:
            action = 'hit'
    
        hit_or_stand(the_deck, player_hand, action)
        
        print("\nPlayer's Hand:", *player_hand.cards, sep='\n')
        print("\nPlayer's Hand =", player_hand.value)
    
        if player_hand.value>21:
            score += 0
            print('BUST!')
            break


    if player_hand.value <= 21:
        score += player_hand.value ** 2
    

    if len(list(the_deck))>4:
        playing = True
        continue
    else:
        print('Not enough cards left in deck!')
        print(f'FINAL SCORE : {score}')
        break

how many decks?1
Your current score is 0

Player's Hand:
Seven of Diamonds
Five of Hearts

Player's Hand = 12

Player's Hand:
Seven of Diamonds
Five of Hearts
King of Clubs

Player's Hand = 22
BUST!
Your current score is 0

Player's Hand:
Jack of Clubs
Ace of Hearts

Player's Hand = 21

Player's Hand:
Jack of Clubs
Ace of Hearts

Player's Hand = 21
Your current score is 441

Player's Hand:
Two of Diamonds
Eight of Clubs

Player's Hand = 10

Player's Hand:
Two of Diamonds
Eight of Clubs
Six of Diamonds

Player's Hand = 16

Player's Hand:
Two of Diamonds
Eight of Clubs
Six of Diamonds
Three of Hearts

Player's Hand = 19

Player's Hand:
Two of Diamonds
Eight of Clubs
Six of Diamonds
Three of Hearts

Player's Hand = 19
Your current score is 802

Player's Hand:
Ten of Hearts
Two of Clubs

Player's Hand = 12

Player's Hand:
Ten of Hearts
Two of Clubs
Seven of Hearts

Player's Hand = 19

Player's Hand:
Ten of Hearts
Two of Clubs
Seven of Hearts

Player's Hand = 19
Your current score is 1163



In [32]:
sums = np.arange(12, 22)
aces = np.arange(0,2)
count = np.arange(-40, 40)
cards_in_deck = np.arange(1, 103)
actions = {1:'hit', 2:'stand'}
# To reduce statespace, we define a 'count average', which is count divided by remaining cards in deck.
count_average = np.linspace(-1, 1, 21)

In [33]:
count_average

array([-1. , -0.9, -0.8, -0.7, -0.6, -0.5, -0.4, -0.3, -0.2, -0.1,  0. ,
        0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ])

In [34]:
statespace = np.array([sums, aces, count_average])

In [35]:
states = []
for i in statespace[0]:
    for j in statespace[1]:
        for k in statespace[2]:
            states.append([i,j,k])

In [36]:
states1 = []
for i in statespace[0]:
    for j in statespace[1]:
        states1.append([i,j])

In [37]:
q_table = pd.DataFrame({'states':states, 'hit':np.zeros(len(states)), 'stand':np.zeros(len(states))})

In [38]:
q1 = pd.DataFrame({'states':states1, 'hit':np.zeros(len(states1)), 'stand':np.zeros(len(states1))})

In [39]:
q_table

,states,hit,stand
0,"[12, 0, -1.0]",0.0,0.0
1,"[12, 0, -0.9]",0.0,0.0
2,"[12, 0, -0.8]",0.0,0.0
3,"[12, 0, -0.7]",0.0,0.0
4,"[12, 0, -0.6]",0.0,0.0
...,...,...,...
415,"[21, 1, 0.6000000000000001]",0.0,0.0
416,"[21, 1, 0.7000000000000002]",0.0,0.0
417,"[21, 1, 0.8]",0.0,0.0
418,"[21, 1, 0.9000000000000001]",0.0,0.0


In [40]:
q1

,states,hit,stand
0,"[12, 0]",0.0,0.0
1,"[12, 1]",0.0,0.0
2,"[13, 0]",0.0,0.0
3,"[13, 1]",0.0,0.0
4,"[14, 0]",0.0,0.0
5,"[14, 1]",0.0,0.0
6,"[15, 0]",0.0,0.0
7,"[15, 1]",0.0,0.0
8,"[16, 0]",0.0,0.0
9,"[16, 1]",0.0,0.0


In [41]:
actions = {1:'hit', 2:'stand'}

In [339]:
hits = np.zeros(len(states1))
stands = np.zeros(len(states1))
iterations = 10000
occurences = np.zeros(len(states1))

for i in range(iterations):

    n = 2
    the_deck = Deck()
    the_deck.shuffle()
    
    while True:    

        player_hand = Hand()

        player_hand.add_card(the_deck.deal())

        player_hand.add_card(the_deck.deal())

        player_hand.adjust_for_ace()
        
        reward_h = 0

        reward_s = 0
        
        previous_sum = 0
        
        while playing:

            if len(list(the_deck)) == 0:
                break 

            action = actions[random.randint(1,2)]
            previous_sum = player_hand.value
            
            hit_or_stand(the_deck, player_hand, action)

            if player_hand.value > 21:
                reward_h = -5
                break

        if player_hand.value <= 21:
            if action == 'hit':
                if player_hand.value >= previous_sum:
                    reward_h = 1
                    reward_s = -5
                    
                else:
                    reward_h = -1
            else:
                reward_s = 0.05
        
        for s in range(len(states1)):
            if previous_sum == states1[s][0]:
                if player_hand.aces == states1[s][1]:
                    hits[s] += reward_h
                    stands[s] += reward_s
                    occurences[s] += 1

        if len(list(the_deck)) > 2:
            playing = True
            continue
        else:
            break

for i in range(len(hits)):
    if occurences[i] != 0:
        hits[i] = hits[i]/occurences[i]
        stands[i] = stands[i]/occurences[i]

In [340]:
q1 = pd.DataFrame({'states':states1, 'hit':hits, 'stand':stands})
q1

,states,hit,stand
0,"[12, 0]",-1.182612,-0.014538
1,"[12, 1]",0.006073,0.019332
2,"[13, 0]",-1.339155,-0.016204
3,"[13, 1]",0.012931,-0.015302
4,"[14, 0]",-1.554089,-0.007004
5,"[14, 1]",0.015728,-0.029429
6,"[15, 0]",-1.718053,0.006331
7,"[15, 1]",0.004000,0.029800
8,"[16, 0]",-1.878763,0.006264
9,"[16, 1]",0.009231,0.003385


In [549]:
q1.to_csv('q-table.csv')

In [341]:
iterations = 10000
scores = 0
total_hands = 0
for i in range(iterations):

    n = 2
    the_deck = Deck()
    the_deck.shuffle()
    score = 0
    hands_played = 0
    
    while True:    

        player_hand = Hand()

        player_hand.add_card(the_deck.deal())

        player_hand.add_card(the_deck.deal())

        player_hand.adjust_for_ace()


        while playing:

            if len(list(the_deck)) == 0:
                break 

            if player_hand.value <= 11:
                action = 'hit'
            
            for s in range(len(states1)):
                if player_hand.value == states1[s][0]:
                    if player_hand.aces == states1[s][1]:
                        if hits[s] >= stands[s]:
                            action = 'hit'
                        else:
                            action = 'stand'
                            hands_played += 1


            hit_or_stand(the_deck, player_hand, action)

            if player_hand.value>21:
                score += 0
                hands_played += 1
                break


        if player_hand.value <= 21:
            score += player_hand.value ** 2


        if len(list(the_deck))>2:
            playing = True
            continue
        else:
            #final score
            break
        
    scores += score
    total_hands += hands_played
print(f'average score:{scores/iterations}') 
print(f'average no. of hands played:{total_hands/iterations}')
print(f'average sum per hand:{(scores/total_hands)**0.5}')

average score:6202.6954
average no. of hands played:21.7361
average sum per hand:16.892713462001137


In [ ]:
# 6202
# 16.89 

In [331]:
hits = np.zeros(len(states))
stands = np.zeros(len(states))
iterations = 10000
occurences = np.zeros(len(states))
h = 4.7
s = 1.7

for i in range(iterations):

    n = 2
    the_deck = Deck()
    the_deck.shuffle()
    
    while True:    

        player_hand = Hand()

        player_hand.add_card(the_deck.deal())

        player_hand.add_card(the_deck.deal())

        player_hand.adjust_for_ace()
        
        reward_h = 0

        reward_s = 0
        
        previous_sum = 0
        
        while playing:

            if len(list(the_deck)) == 0:
                break 

            action = actions[random.randint(1,2)]
            previous_sum = player_hand.value
            previous_length = len(list(the_deck))
            previous_count = the_deck.count
            
            hit_or_stand(the_deck, player_hand, action)

            if player_hand.value > 21:
                reward_h = -5 * h
                break

        if player_hand.value <= 21:
            if action == 'hit':
                if player_hand.value >= previous_sum:
                    reward_h = 1 * h
                    reward_s = -5 * s
                    
                else:
                    reward_h = -1 * h
            else:
                reward_s = 0.05 * s
        
        for s in range(len(states)):
            if previous_sum == states[s][0]:
                if player_hand.aces == states[s][1]:
                    if abs(previous_count/previous_length - states[s][2]) < 0.04499:
                        hits[s] += reward_h
                        stands[s] += reward_s
                        occurences[s] += 1

        if len(list(the_deck)) > 2:
            playing = True
            continue
        else:
            break

for i in range(len(hits)):
    if occurences[i] != 0:
        hits[i] = hits[i]/occurences[i]
        stands[i] = stands[i]/occurences[i]

In [332]:
q_table = pd.DataFrame({'states':states, 'hit':hits, 'stand':stands})

In [333]:
q_table

,states,hit,stand
0,"[12, 0, -1.0]",-0.575510,-585.317347
1,"[12, 0, -0.9]",-4.406250,17.021875
2,"[12, 0, -0.8]",-2.350000,18.855000
3,"[12, 0, -0.7]",-4.351852,17.070370
4,"[12, 0, -0.6]",-1.958333,19.204167
...,...,...,...
415,"[21, 1, 0.6000000000000001]",0.000000,20.950000
416,"[21, 1, 0.7000000000000002]",0.000000,20.950000
417,"[21, 1, 0.8]",0.000000,20.950000
418,"[21, 1, 0.9000000000000001]",0.000000,20.950000


In [335]:
q_table.to_csv('q_table.csv')

In [334]:
iterations = 10000
scores = 0
total_hands = 0
for i in range(iterations):

    n = 2
    the_deck = Deck()
    the_deck.shuffle()
    score = 0
    hands_played = 0
    
    while True:    

        player_hand = Hand()

        player_hand.add_card(the_deck.deal())

        player_hand.add_card(the_deck.deal())

        player_hand.adjust_for_ace()


        while playing:

            if len(list(the_deck)) == 0:
                break 

            if player_hand.value <= 11:
                action = 'hit'
            
            for s in range(len(states)):
                if player_hand.value == states[s][0]:
                    if player_hand.aces == states[s][1]:
                        if abs((the_deck.count/len(list(the_deck))) - states[s][2]) < 0.04499:
                            if hits[s] >= stands[s]:
                                action = 'hit'
                            else:
                                action = 'stand'
                                hands_played += 1


            hit_or_stand(the_deck, player_hand, action)

            if player_hand.value>21:
                score += 0
                hands_played += 1
                break


        if player_hand.value <= 21:
            score += player_hand.value ** 2


        if len(list(the_deck))>2:
            playing = True
            continue
        else:
            #final score
            break
        
    scores += score
    total_hands += hands_played
print(f'average score:{scores/iterations}') 
print(f'average no. of hands played:{total_hands/iterations}')
print(f'average sum per hand:{(scores/total_hands)**0.5}')

average score:5920.8626
average no. of hands played:20.024
average sum per hand:17.195589668143334


In [ ]:
# 17.17 with original reward values
# 17.2 with optimised reward values - negligible difference

In [319]:
# Trying different reward values - This cell takes a very long time to run!
r_h = np.arange(3.5,5,0.1)

r_s = np.arange(1.5,3.1, 0.1)

scores_rewards = []
for h in r_h:
    for t in r_s:
        
        hits = np.zeros(len(states))
        stands = np.zeros(len(states))
        iterations = 1000
        occurences = np.zeros(len(states))

        for i in range(iterations):

            n = 2
            the_deck = Deck()
            the_deck.shuffle()

            while True:    

                player_hand = Hand()

                player_hand.add_card(the_deck.deal())

                player_hand.add_card(the_deck.deal())

                player_hand.adjust_for_ace()
                
                reward_h = 0
                
                reward_s = 0

                previous_sum = 0

                while playing:

                    if len(list(the_deck)) == 0:
                        break 

                    action = actions[random.randint(1,2)]
                    previous_sum = player_hand.value
                    previous_length = len(list(the_deck))
                    previous_count = the_deck.count

                    hit_or_stand(the_deck, player_hand, action)

                    if player_hand.value > 21:
                        reward_h = -5 * h
                        break

                if player_hand.value <= 21:
                    if action == 'hit':
                        if player_hand.value >= previous_sum:
                            reward_h = h
                            reward_s = -5 * t

                        else:
                            reward_h = -h
                    else:
                        reward_s = 0.05 * t

                for s in range(len(states)):
                    if previous_sum == states[s][0]:
                        if player_hand.aces == states[s][1]:
                            if abs(previous_count/previous_length - states[s][2]) < 0.04499:
                                hits[s] += reward_h
                                stands[s] += reward_s
                                occurences[s] += 1

                if len(list(the_deck)) > 2:
                    playing = True
                    continue
                else:
                    break

        for i in range(len(hits)):
            if occurences[i] != 0:
                hits[i] = hits[i]/occurences[i]
                stands[i] = stands[i]/occurences[i]

        scores = 0
        total_hands = 0
        for i in range(iterations):

            n = 2
            the_deck = Deck()
            the_deck.shuffle()
            score = 0
            hands_played = 0

            while True:    

                player_hand = Hand()

                player_hand.add_card(the_deck.deal())

                player_hand.add_card(the_deck.deal())

                player_hand.adjust_for_ace()


                while playing:

                    if len(list(the_deck)) == 0:
                        break 

                    if player_hand.value <= 11:
                        action = 'hit'

                    for s in range(len(states)):
                        if player_hand.value == states[s][0]:
                            if player_hand.aces == states[s][1]:
                                if abs((the_deck.count/len(list(the_deck))) - states[s][2]) < 0.04499:
                                    if hits[s] >= stands[s]:
                                        action = 'hit'
                                    else:
                                        action = 'stand'
                                        hands_played += 1


                    hit_or_stand(the_deck, player_hand, action)

                    if player_hand.value>21:
                        score += 0
                        hands_played += 1
                        break


                if player_hand.value <= 21:
                    score += player_hand.value ** 2


                if len(list(the_deck))>2:
                    playing = True
                    continue
                else:
                    #final score
                    break

            scores += score
            total_hands += hands_played
        scores_rewards.append([scores/iterations, h, t])


In [318]:
max(scores_rewards)

[5975.86, 4.700000000000001, 1.7000000000000002]

In [ ]:
# 4.7, 1.7